In [3]:
# paths must reflect the layout above
from pathlib import Path
assert Path('FairFace/train').is_dir(),        "Image folder not found"
assert Path('FairFace/fairface_label_train.csv').is_file(), "CSV missing"

import pandas as pd
train_df = pd.read_csv('Datasets/trainingSet.csv')
val_df   = pd.read_csv('Datasets/validationSet.csv')

print("Train  shape:", train_df.shape)
print("Val    shape:", val_df.shape)
print("\nFirst few rows:")
display(train_df.head())

print("\nGrouped counts (age × gender):")
print(train_df.groupby(['age_label','gender_label']).size().unstack(fill_value=0))


Train  shape: (2240, 4)
Val    shape: (560, 4)

First few rows:


,file,age_label,gender_label,race_label
0,train/40251.jpg,7,1,5
1,train/72678.jpg,1,0,5
2,train/57259.jpg,7,1,3
3,train/35937.jpg,4,0,0
4,train/84508.jpg,1,0,5



Grouped counts (age × gender):
gender_label    0    1
age_label             
0             140  140
1             140  140
2             140  140
3             140  140
4             140  140
5             140  140
6             140  140
7             140  140


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

# ------------------------------------------------------------------
# Dummy dataset that yields:
#   • an RGB tensor (3 × 224 × 224) with random values
#   • a dict of two labels: {"age": LongTensor, "gender": LongTensor}
# ------------------------------------------------------------------
class DummyFairFace(Dataset):
    def __init__(self, n_samples=1_000, img_size=224, n_age=9):
        self.n_samples = n_samples
        self.img_size  = img_size
        self.n_age     = n_age          # 9 age classes (0‥8)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        # random image (values ~ N(0,1))
        img = torch.randn(3, self.img_size, self.img_size)

        # random labels matching your real heads
        age_label    = torch.randint(0, self.n_age, (1,), dtype=torch.long)
        gender_label = torch.randint(0, 2,        (1,), dtype=torch.long)

        return img, {"age": age_label.squeeze(), "gender": gender_label.squeeze()}

# ------------------------------------------------------------------
# Build loaders the same way you do for the real set
# ------------------------------------------------------------------
train_ds = DummyFairFace(n_samples=2_048)   # 2048 fake samples
val_ds   = DummyFairFace(n_samples=512)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=0)

# quick sanity-check
x, tgt = next(iter(train_loader))
print("batch shapes:", x.shape, tgt["age"].shape, tgt["gender"].shape)


batch shapes: torch.Size([64, 3, 224, 224]) torch.Size([64]) torch.Size([64])


In [6]:
from torch.utils.data import DataLoader
import time, torch

def loader_speed(dl, n_iter=50):
    t0 = time.time()
    for i, _ in zip(range(n_iter), dl):
        pass
    return (time.time() - t0) / n_iter    # sec per batch

for w in [0, 2, 4, 8]:
    dl = DataLoader(train_ds, batch_size=64,
                    shuffle=True, num_workers=w, pin_memory=True)
    print(f"workers={w}: {loader_speed(dl):.3f}s  per batch")


workers=0: 0.031s  per batch


RuntimeError: DataLoader worker (pid(s) 52344, 61336) exited unexpectedly

In [42]:
import torch, torch.nn as nn, torchvision.models as models
from torchvision import transforms
from PIL import Image
from pathlib import Path

# ── adjust paths ─────────────────────────────────────────────
CKPT_FILE = r"checkpoints/model_epoch_100.pth"     # pick any epoch file
TEST_IMG  = r"Datasets\testingImages\asian3.jpg"          # image to demo

# class-name lists (same order you used when encoding)
age_classes  = ['0-2','3-9','10-19','20-29','30-39',
                '40-49','50-59','60-69','70+']
gender_map   = ['Female','Male']
race_classes = ['White','Black','Hispanic','East Asian',
                'Southeast Asian','Indian','Middle Eastern']

# ── 1. recreate your ResNet-18 multitask model ───────────────
class AgeSexRaceNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet18(weights=None)
        self.feats = nn.Sequential(*(list(resnet.children())[:-1]))  # 512-d
        self.head_age   = nn.Linear(512, len(age_classes))
        self.head_gen   = nn.Linear(512, len(gender_map))
        self.head_race  = nn.Linear(512, len(race_classes))
    def forward(self, x):
        x = self.feats(x).flatten(1)
        return {"age": self.head_age(x),
                "gender": self.head_gen(x),
                "race": self.head_race(x)}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = AgeSexRaceNet().to(device)

# ── 2. load state-dict directly ──────────────────────────────
state_dict = torch.load(CKPT_FILE, map_location=device)
model.eval()
print(f"✓  loaded weights from {CKPT_FILE}")

# ── 3. validation / inference transform (same as training) ──
IMG_SIZE = 224

infer_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

@torch.inference_mode()
def predict(img_path: str):
    img = Image.open(img_path).convert("RGB")
    x   = infer_tfms(img).unsqueeze(0).to(device)
    out = model(x)
    return {
        "age_range": age_classes[out["age"].argmax(1).item()],
        "gender"   : gender_map[out["gender"].argmax(1).item()],
        "race"     : race_classes[out["race"].argmax(1).item()]
    }

print("Prediction →", predict(TEST_IMG))


✓  loaded weights from checkpoints/model_epoch_100.pth
Prediction → {'age_range': '30-39', 'gender': 'Male', 'race': 'Indian'}


C:\Users\User\AppData\Local\Temp\ipykernel_27364\2408865314.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(CKPT_FILE, map_location=device)
